In [1]:
import numpy as np
import os
import timeit
import glob 
import pandas as pd
import time
import multiprocessing as mp 
import logging
import sys 
import pathlib
from pathlib import Path
import matplotlib.pyplot as plt 

Matplotlib created a temporary config/cache directory at /scratch/slurm-job.871838/matplotlib-xr3ba5k0 because the default path (/cluster/customapps/biomed/grlab/users/prelotla/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Matplotlib is building the font cache; this may take a moment.


In [2]:
def get_junction_coordinates(df, coordinates_col, sep=':'):
    df['strand'] = None
    df['junction_coordinate'] = None

    for idx, row in df.iterrows():
        kmer_coordinates = [int(x) for x in row[coordinates_col].split(sep) if (x !='None') ]

        if kmer_coordinates[1] < kmer_coordinates[2]: # order strand +

            df.loc[idx, 'strand'] = '+'
            if len(kmer_coordinates) == 4:  # 2 exons
                df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in kmer_coordinates[1:3]])
            elif len(kmer_coordinates) == 6:
                df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in kmer_coordinates[1:5]])
        else: # order strand +
            df.loc[idx, 'strand'] = '-'
            if len(kmer_coordinates) == 4:  # 2 exons
                df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in [kmer_coordinates[3],
                                                                                kmer_coordinates[0]]])
            elif len(kmer_coordinates) == 6:
                df.loc[idx, 'junction_coordinate'] = ':'.join([str(x) for x in [kmer_coordinates[3],
                                                                                kmer_coordinates[0],
                                                                                kmer_coordinates[2],
                                                                                kmer_coordinates[5]
                                                                               ]])
    return df




### Get filtered data

In [8]:
# Inputs
run_type = 'ov'

if run_type == 'brca':
    target_samples = ['TCGA-C8-A12P-01A-11R-A115-07.all',
                      'TCGA-AO-A0JM-01A-21R-A056-07.all',
                      'TCGA-BH-A18V-01A-11R-A12D-07.all',
                      'TCGA-A2-A0D2-01A-21R-A034-07.all',
                      'TCGA-A2-A0SX-01A-12R-A084-07.all']
    sample_target = 'TCGA-AO-A0JM-01A-21R-A056-07'
    basedir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102'
    intermediate_output = os.path.join(basedir, 'filtering_intermediate/complete_cancer_candidates_order_r.tsv.gz')
elif run_type == 'ov':
    target_samples = ['TCGA-25-1319-01A-01R-1565-13.all',
                      'TCGA-25-1313-01A-01R-1565-13.all',
                      'TCGA-61-2008-01A-02R-1568-13.all',
                      'TCGA-24-1431-01A-01R-1566-13.all',
                      'TCGA-24-2298-01A-01R-1569-13.all']
    sample_target = 'TCGA-25-1319-01A-01R-1565-13'
    basedir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374'
    intermediate_output = os.path.join(basedir, 'filtering_intermediate/complete_cancer_candidates_order_r.tsv.gz')


# Outputs
filtering_id = 'filters_22March_order_wany_wAnnot'
output_dir = os.path.join(basedir, 'filtering_samples', filtering_id)

In [4]:
# Load generation matrix
df_load = pd.read_csv(intermediate_output, sep = '\t')

In [9]:
# Load filtered kmers
path_interest = f'G_{sample_target}_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1.tsv.gz'
path_interest = os.path.join(output_dir, path_interest)
filt = pd.read_csv(path_interest, sep = '\t')
print(filt.shape)

(131575, 4)


In [10]:
# Merge generation matrix & filtered kmers
filt_meta = df_load.merge(filt, on = list(filt.columns), how = 'right')
print(filt_meta.shape)

(131962, 24)


In [11]:
# Define problematic kmer
# exp_lim = 600
# filt_meta_pb = filt_meta.loc[filt_meta['cancerCohortfilter >0.0'] > exp_lim]
# filt_meta_pb = filt_meta_pb[filt_meta_pb[sample_target.replace('-', '') + 'all'] > 0 ]
# print(f'filtered kmers-junctions {filt_meta.shape[0]}')
# print(f'filtered kmers unique {len(filt_meta.kmer.unique())}')
# print(f'filtered kmers-junctions problematic {filt_meta_pb.shape[0]}')
# print(f'filtered kmers problematic unique {len(filt_meta_pb.kmer.unique())}')
filt_meta_pb = filt_meta

In [12]:
# Columns magic
core_cols = ['kmer', 'gtexCohortfilter >0.0', 'coord',
       'junctionAnnotated', 'readFrameAnnotated', 'isCrossJunction', 'batch',
       'cancerCohortfilter >0.0', sample_target.replace('-', '') + 'all', 'isAnnotated']

df_load.columns

Index(['kmer', 'gtexCohortfilter >0.0', 'gtexCohortfilter >=1.0',
       'gtexCohortfilter >=2.0', 'gtexCohortfilter >=3.0',
       'gtexCohortfilter >=5.0', 'gtexCohortfilter >=10.0', 'coord',
       'junctionAnnotated', 'readFrameAnnotated', 'isCrossJunction', 'batch',
       'cancerCohortfilter >0.0', 'cancerCohortfilter >=1.0',
       'cancerCohortfilter >=2.0', 'cancerCohortfilter >=3.0',
       'cancerCohortfilter >=5.0', 'cancerCohortfilter >=10.0',
       'TCGA25131901A01R156513all', 'TCGA25131301A01R156513all',
       'TCGA61200801A02R156813all', 'TCGA24143101A01R156613all',
       'TCGA24229801A01R156913all', 'isAnnotated'],
      dtype='object')

In [13]:
# Extract coordinates of the merged table
filt_meta_pb = get_junction_coordinates(filt_meta_pb, 'coord', sep=':')

display(filt_meta_pb[core_cols + ['junction_coordinate']].head())

,kmer,gtexCohortfilter >0.0,coord,junctionAnnotated,readFrameAnnotated,isCrossJunction,batch,cancerCohortfilter >0.0,TCGA25131901A01R156513all,isAnnotated,junction_coordinate
0,RMREATKGP,NaN,55350885:55350897:55352540:55352555:None:None,False,True,True,16543,2,10.91096,NaN,55350897:55352540
1,QVERMREAT,NaN,55350876:55350897:55352540:55352546:None:None,False,True,True,16543,2,10.91096,NaN,55350897:55352540
2,VERMREATK,NaN,55350879:55350897:55352540:55352549:None:None,False,True,True,16543,2,10.91096,NaN,55350897:55352540
3,MREATKGPV,NaN,55350888:55350897:55352540:55352558:None:None,False,True,True,16543,2,10.91096,NaN,55350897:55352540
4,VQVERMREA,NaN,55350873:55350897:55352540:55352543:None:None,False,True,True,16543,2,10.91096,NaN,55350897:55352540


### Add the peptide metadata

In [14]:
# Define peptide file columns of interest 
cols_correct = ['peptide','id','readFrame','readFrameAnnotated','geneName','geneChr','geneStrand',
'mutationMode','hasStopCodon','isInJunctionList',
'isIsolated','variantComb','variantSegExpr','modifiedExonsCoord',
'originalExonsCoord',
'vertexIdx','kmerType', 'dummy1', 'dummy2', 'dummy3']

cols_pep_file = ['peptide', 'id', 'readFrame', 'geneName',
       'geneChr', 'geneStrand', 'mutationMode',
       'hasStopCodon', 'isInJunctionList', 'isIsolated',
       'variantSegExpr', 'modifiedExonsCoord', 'originalExonsCoord',
       'vertexIdx', 'kmerType']


cols_pep_file = {col:  'gtex_' + col for col in cols_pep_file}


In [15]:
len(filt_meta_pb['batch'].unique()) # Now 121 brca, # 2792 OV

2792

In [16]:
filt_meta_pb.head()

,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,cancerCohortfilter >=5.0,cancerCohortfilter >=10.0,TCGA25131901A01R156513all,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated,strand,junction_coordinate
0,RMREATKGP,NaN,NaN,NaN,NaN,NaN,NaN,55350885:55350897:55352540:55352555:None:None,False,True,...,2,2,10.91096,0.0,0.0,0.0,0.0,NaN,+,55350897:55352540
1,QVERMREAT,NaN,NaN,NaN,NaN,NaN,NaN,55350876:55350897:55352540:55352546:None:None,False,True,...,2,2,10.91096,0.0,0.0,0.0,0.0,NaN,+,55350897:55352540
2,VERMREATK,NaN,NaN,NaN,NaN,NaN,NaN,55350879:55350897:55352540:55352549:None:None,False,True,...,2,2,10.91096,0.0,0.0,0.0,0.0,NaN,+,55350897:55352540
3,MREATKGPV,NaN,NaN,NaN,NaN,NaN,NaN,55350888:55350897:55352540:55352558:None:None,False,True,...,2,2,10.91096,0.0,0.0,0.0,0.0,NaN,+,55350897:55352540
4,VQVERMREA,NaN,NaN,NaN,NaN,NaN,NaN,55350873:55350897:55352540:55352543:None:None,False,True,...,2,2,10.91096,0.0,0.0,0.0,0.0,NaN,+,55350897:55352540


### Make False positive hypothesis

In [17]:
gtex_dir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref'
check_hypothesis = []
kmers_not_in_gtex = []
kmers_no_metadata = []

for batch_i, batch in enumerate(filt_meta_pb['batch'].unique()):
    print(f'\n Iteration {batch_i} batch {batch}')
    # CANCER SIDE GET BATCH Junctions
    batch_kmers_init = filt_meta_pb.loc[filt_meta_pb['batch'] == batch]
    print(f'Size cancer kmers-junctions {len(batch_kmers_init)}')
    
    # Read GTEX peptide file
    pep_path = os.path.join(gtex_dir, 'cohort_mutNone', f'tmp_out_ref_batch_{batch}', 'ref_sample_peptides_meta.gz')
    if os.path.exists(pep_path):
        ##### COPY 
        not_in_background = False


        print(pep_path)
        meta_pep = pd.read_csv(pep_path, sep = '\t')
        print(meta_pep.shape)
        meta_pep.columns = cols_correct # ISSUE WITH IMMUNOPEPPER



        # Only bi or tri-exons peptides
        jx_ids = [idx for idx, coord in enumerate(meta_pep['modifiedExonsCoord']) if 'nan' not in coord]
        meta_pep = meta_pep.iloc[jx_ids]

        # Quick Assess Cancer junction presence in GTEX
        coord_int = [i.split(':') for i in batch_kmers_init['junction_coordinate'].unique()]
        keep = set()
        for cd in coord_int:
            keep.update([idx for idx, coord in enumerate(meta_pep['modifiedExonsCoord']) \
                         if (cd[0] in coord) and (cd[1] in coord)])
        meta_pep = meta_pep.iloc[list(keep)]

        if keep: # Some target junctions are in "potentially" found in gtex 
            # Rename
            meta_pep = meta_pep.rename(cols_pep_file, axis = 1)

            # Extract coordinates peptides
            meta_pep = get_junction_coordinates(meta_pep, 'gtex_modifiedExonsCoord', sep=';')

            # Add Peptide info 
            meta_pep = meta_pep[list(cols_pep_file.values()) + ['junction_coordinate']].drop_duplicates()
            
            exist_jx_not_in_gtex = set(batch_kmers_init['junction_coordinate']).difference(set(meta_pep['junction_coordinate']))
            exist_jx_in_gtex = set(batch_kmers_init['junction_coordinate']).intersection(set(meta_pep['junction_coordinate']))

            if exist_jx_in_gtex: # Some target junctions are in gtex
                batch_kmers = batch_kmers_init.merge(meta_pep, on = 'junction_coordinate', how = 'inner')
                print(f'Size cancer kmers-junctions + all GTEX peptides {len(batch_kmers)}')

                # Calculate the number of aa fitting on each side of the junctions
                batch_kmers['gtex_aa_E1'] = batch_kmers['gtex_modifiedExonsCoord'].str.split(';').map(lambda x: abs(int(x[0]) - int(x[1]))/3 )
                batch_kmers['gtex_aa_E2'] = batch_kmers['gtex_modifiedExonsCoord'].str.split(';').map(lambda x: abs(int(x[2]) - int(x[3]))/3 )
                batch_kmers['cancer_aa_E1'] = batch_kmers['coord'].str.split(':').map(lambda x: abs(int(x[0]) - int(x[1]))/3 )
                batch_kmers['cancer_aa_E2'] = batch_kmers['coord'].str.split(':').map(lambda x: abs(int(x[2]) - int(x[3]))/3 )
                # What about 3 exons?

                # Validate hypothesis: 
                # H1: The second exon is not long enough to get the translation through in GTEX. BUT no exon was added on the right
                new_col = 'gtexE2<cancE2'
                batch_kmers[new_col] = True
                batch_kmers.loc[batch_kmers['gtex_aa_E2'] > batch_kmers['cancer_aa_E2'], new_col] = False

                res = batch_kmers[['kmer', new_col, 'junction_coordinate' ]].drop_duplicates()
                display( batch_kmers[['kmer', 'cancerCohortfilter >0.0',\
                                      sample_target.replace('-', '') + 'all',
                                      'readFrameAnnotated', \
                                      'junctionAnnotated', \
                                      new_col ]].drop_duplicates() )
                check_hypothesis.append(res[['kmer', 'junction_coordinate']])
                
            if exist_jx_not_in_gtex: # Some target junctions are NOT in gtex
                not_in_background = True
                diff = batch_kmers_init.set_index('junction_coordinate').loc[exist_jx_not_in_gtex].reset_index() #left anti join

        
        else: # No target junctions are in GTEX at all
            not_in_background = True
            diff = batch_kmers_init.copy()

        if not_in_background:
            #diff = diff[['kmer', 'junction_coordinate', 'cancerCohortfilter >0.0', ]].drop_duplicates()
            kmers_not_in_gtex.append(diff[['kmer', 'junction_coordinate']])
            print(f'{diff.shape[0]} Kmers - junctions not found in gtex. Recurrence is:')
            print(diff['cancerCohortfilter >0.0'].unique())
            for rec in diff['cancerCohortfilter >0.0'].unique():
                if rec > 100: 
                    display( diff[['kmer', 'junction_coordinate', 'coord', \
                                   'cancerCohortfilter >0.0',\
                                      sample_target.replace('-', '') + 'all',
                                      'readFrameAnnotated', \
                                      'junctionAnnotated' ]].drop_duplicates() )
            print(f'{diff.shape[0]} Kmers - junctions not found in gtex. Junction annotated is:')
            print(diff['junctionAnnotated'].unique())
            print(f'{diff.shape[0]} Kmers - junctions not found in gtex. RF annotated is:')
            print(diff['readFrameAnnotated'].unique())

            
        ##### END COPY 
    else:
        check_path = os.path.join(gtex_dir, 'cohort_mutNone', f'tmp_out_ref_batch_{batch}')
        kmers_no_metadata.append(batch_kmers_init[['kmer', 'junction_coordinate']])
        for rec in diff['cancerCohortfilter >0.0'].unique():
            if rec > 1000: 
                display(batch_kmers_init[['kmer', 'junction_coordinate']])
        print(f'CHECK COMPLETION OF {check_path}')
        continue
#         for pep_path in glob.glob(pep_path + '/*'):
           




 Iteration 0 batch 16543
Size cancer kmers-junctions 7
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_16543/ref_sample_peptides_meta.gz
(15185, 20)
7 Kmers - junctions not found in gtex. Recurrence is:
[2]
7 Kmers - junctions not found in gtex. Junction annotated is:
[False]
7 Kmers - junctions not found in gtex. RF annotated is:
[ True]

 Iteration 1 batch 12597
Size cancer kmers-junctions 4
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_12597/ref_sample_peptides_meta.gz
(31021, 20)
4 Kmers - junctions not found in gtex. Recurrence is:
[4]
4 Kmers - junctions not found in gtex. Junction annotated is:
[False]
4 Kmers - junctions not found in gtex. RF annotated is:
[False]

 Iteration 2 batch 41047
Size cancer kmers-junctions 6
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generati

,kmer,cancerCohortfilter >0.0,TCGA25131901A01R156513all,readFrameAnnotated,junctionAnnotated,gtexE2<cancE2
0,SRIPCPSQL,33,5.875132,False,False,True
9,RIPCPSQLP,33,5.875132,False,False,True
18,GSPAPHSCR,33,5.875132,False,False,True
27,IPCPSQLPA,33,5.875132,False,False,True
36,SRIPCPSQN,143,5.875132,False,True,True
81,IPCPSQNTS,143,5.875132,False,True,True
126,PGSPAPHRI,143,5.875132,False,True,True
171,GSPAPHRIH,143,5.875132,False,True,True
216,RIPCPSQNT,143,5.875132,False,True,True
261,IPCPSQEKV,20,1.678609,False,False,True


87 Kmers - junctions not found in gtex. Recurrence is:
[ 5 12 24 16  4 26 89  8]
87 Kmers - junctions not found in gtex. Junction annotated is:
[False  True]
87 Kmers - junctions not found in gtex. RF annotated is:
[False]

 Iteration 4 batch 416
Size cancer kmers-junctions 19
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_416/ref_sample_peptides_meta.gz
(33104, 20)
19 Kmers - junctions not found in gtex. Recurrence is:
[17 14  4]
19 Kmers - junctions not found in gtex. Junction annotated is:
[False  True]
19 Kmers - junctions not found in gtex. RF annotated is:
[False]

 Iteration 5 batch 27568
Size cancer kmers-junctions 122
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_27568/ref_sample_peptides_meta.gz
(11126, 20)
122 Kmers - junctions not found in gtex. Recurrence is:
[88 20 12 28 44 42

,kmer,junction_coordinate,coord,cancerCohortfilter >0.0,TCGA25131901A01R156513all,readFrameAnnotated,junctionAnnotated
0,MLRKERELR,35697806:35697872,35697872:35697895:35697802:35697806:None:None,10,0.839305,False,False
1,KERELRDEH,35697806:35697872,35697872:35697886:35697793:35697806:None:None,10,0.839305,False,False
2,LRKERELRD,35697806:35697872,35697872:35697892:35697799:35697806:None:None,10,0.839305,False,False
3,RKERELRDE,35697806:35697872,35697872:35697889:35697796:35697806:None:None,10,0.839305,False,False
4,LQKPSSSRL,35699382:35699443,35699443:35699458:35699370:35699382:None:None,8,0.839305,False,False
...,...,...,...,...,...,...,...
133,MLHRFLILS,35697441:35697888,35697888:35697895:35697421:35697441:None:None,18,0.839305,False,False
134,EMLHRFLIL,35697441:35697888,35697888:35697898:35697424:35697441:None:None,18,0.839305,False,False
135,QEEMLHRFL,35697441:35697888,35697888:35697904:35697430:35697441:None:None,18,0.839305,False,False
136,STVLRISRW,35720121:35720206,35720206:35720218:35720106:35720121:None:None,6,0.839305,False,False


,kmer,junction_coordinate,coord,cancerCohortfilter >0.0,TCGA25131901A01R156513all,readFrameAnnotated,junctionAnnotated
0,MLRKERELR,35697806:35697872,35697872:35697895:35697802:35697806:None:None,10,0.839305,False,False
1,KERELRDEH,35697806:35697872,35697872:35697886:35697793:35697806:None:None,10,0.839305,False,False
2,LRKERELRD,35697806:35697872,35697872:35697892:35697799:35697806:None:None,10,0.839305,False,False
3,RKERELRDE,35697806:35697872,35697872:35697889:35697796:35697806:None:None,10,0.839305,False,False
4,LQKPSSSRL,35699382:35699443,35699443:35699458:35699370:35699382:None:None,8,0.839305,False,False
...,...,...,...,...,...,...,...
133,MLHRFLILS,35697441:35697888,35697888:35697895:35697421:35697441:None:None,18,0.839305,False,False
134,EMLHRFLIL,35697441:35697888,35697888:35697898:35697424:35697441:None:None,18,0.839305,False,False
135,QEEMLHRFL,35697441:35697888,35697888:35697904:35697430:35697441:None:None,18,0.839305,False,False
136,STVLRISRW,35720121:35720206,35720206:35720218:35720106:35720121:None:None,6,0.839305,False,False


...

,kmer,junction_coordinate,coord,cancerCohortfilter >0.0,TCGA25131901A01R156513all,readFrameAnnotated,junctionAnnotated
16796,LPAPMAEAA,129314988:129315872,129315872:129315878:129314967:129314988:None:None,148,4.196523,True,False
16797,KAARPSPAS,129315296:129315341,129315341:129315358:129315286:129315296:None:None,87,0.839305,False,False
16798,LAKIYTEAA,129314973:129315683,129315683:129315704:129314967:129314973:None:None,123,1.678609,True,False
16799,PRTKRRRAL,129315380:129315464,129315464:129315484:129315373:129315380:None:None,82,1.678609,False,False
16800,RKKLEGAGG,129315328:129315532,129315532:129315551:129315320:129315328:None:None,33,0.839305,True,False
...,...,...,...,...,...,...,...
17025,KIYTEAKKA,129315464:129315674,129315674:129315698:129315461:129315464:None:None,65,59.590629,True,False
17026,KKAAPAHRA,129314973:129315464,129315464:129315470:129314952:129314973:None:None,67,0.839305,False,False
17027,SIKARGDAE,129314928:129315620,129315620:129315629:129314910:129314928:None:None,24,0.839305,False,False
17028,AGGKKAAHP,129314910:129315314,129315314:129315329:129314898:129314910:None:None,17,10.071656,False,False


,kmer,junction_coordinate,coord,cancerCohortfilter >0.0,TCGA25131901A01R156513all,readFrameAnnotated,junctionAnnotated
16796,LPAPMAEAA,129314988:129315872,129315872:129315878:129314967:129314988:None:None,148,4.196523,True,False
16797,KAARPSPAS,129315296:129315341,129315341:129315358:129315286:129315296:None:None,87,0.839305,False,False
16798,LAKIYTEAA,129314973:129315683,129315683:129315704:129314967:129314973:None:None,123,1.678609,True,False
16799,PRTKRRRAL,129315380:129315464,129315464:129315484:129315373:129315380:None:None,82,1.678609,False,False
16800,RKKLEGAGG,129315328:129315532,129315532:129315551:129315320:129315328:None:None,33,0.839305,True,False
...,...,...,...,...,...,...,...
17025,KIYTEAKKA,129315464:129315674,129315674:129315698:129315461:129315464:None:None,65,59.590629,True,False
17026,KKAAPAHRA,129314973:129315464,129315464:129315470:129314952:129314973:None:None,67,0.839305,False,False
17027,SIKARGDAE,129314928:129315620,129315620:129315629:129314910:129314928:None:None,24,0.839305,False,False
17028,AGGKKAAHP,129314910:129315314,129315314:129315329:129314898:129314910:None:None,17,10.071656,False,False


,kmer,junction_coordinate,coord,cancerCohortfilter >0.0,TCGA25131901A01R156513all,readFrameAnnotated,junctionAnnotated
16796,LPAPMAEAA,129314988:129315872,129315872:129315878:129314967:129314988:None:None,148,4.196523,True,False
16797,KAARPSPAS,129315296:129315341,129315341:129315358:129315286:129315296:None:None,87,0.839305,False,False
16798,LAKIYTEAA,129314973:129315683,129315683:129315704:129314967:129314973:None:None,123,1.678609,True,False
16799,PRTKRRRAL,129315380:129315464,129315464:129315484:129315373:129315380:None:None,82,1.678609,False,False
16800,RKKLEGAGG,129315328:129315532,129315532:129315551:129315320:129315328:None:None,33,0.839305,True,False
...,...,...,...,...,...,...,...
17025,KIYTEAKKA,129315464:129315674,129315674:129315698:129315461:129315464:None:None,65,59.590629,True,False
17026,KKAAPAHRA,129314973:129315464,129315464:129315470:129314952:129314973:None:None,67,0.839305,False,False
17027,SIKARGDAE,129314928:129315620,129315620:129315629:129314910:129314928:None:None,24,0.839305,False,False
17028,AGGKKAAHP,129314910:129315314,129315314:129315329:129314898:129314910:None:None,17,10.071656,False,False


234 Kmers - junctions not found in gtex. Junction annotated is:
[False]
234 Kmers - junctions not found in gtex. RF annotated is:
[ True False]

 Iteration 407 batch 32088
Size cancer kmers-junctions 2
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_32088/ref_sample_peptides_meta.gz
(283275, 20)
2 Kmers - junctions not found in gtex. Recurrence is:
[3]
2 Kmers - junctions not found in gtex. Junction annotated is:
[False]
2 Kmers - junctions not found in gtex. RF annotated is:
[False]

 Iteration 408 batch 41780
Size cancer kmers-junctions 6
/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_c4dd02c_conf2_RFall_ref/cohort_mutNone/tmp_out_ref_batch_41780/ref_sample_peptides_meta.gz
(19633, 20)
6 Kmers - junctions not found in gtex. Recurrence is:
[3]
6 Kmers - junctions not found in gtex. Junction annotated is:
[False]
6 Kmers - junctions not found in gt

EmptyDataError: No columns to parse from file

In [18]:
print(filt_meta_pb.shape)
print(filt_meta_pb[['kmer', 'junction_coordinate']].drop_duplicates().shape)

(131962, 26)
(119786, 2)


In [19]:
df_kmers_no_metadata = pd.concat(kmers_no_metadata, axis = 0 )
print(f'{df_kmers_no_metadata.shape[0]} kmers without metadata')
print(f'{df_kmers_no_metadata.drop_duplicates().shape[0]} UNIQUE kmers without metadata')

506 kmers without metadata
434 UNIQUE kmers without metadata


In [20]:
df_kmers_not_in_gtex = pd.concat(kmers_not_in_gtex, axis = 0 )
print(f'{df_kmers_not_in_gtex.shape[0]} kmers completely absent in GTEX')
print(f'{df_kmers_not_in_gtex.drop_duplicates().shape[0]} UNIQUE kmers completely absent in GTEX')

16724 kmers completely absent in GTEX
14990 UNIQUE kmers completely absent in GTEX


In [21]:
df_check_hypothesis = pd.concat(check_hypothesis, axis = 0 )
print(f'{df_check_hypothesis.shape[0]} kmers coming from novel exons')
print(f'{df_check_hypothesis.drop_duplicates().shape[0]} UNIQUE kmers coming from novel exons')

126 kmers coming from novel exons
125 UNIQUE kmers coming from novel exons


In [22]:
# Create new labels based on the false positive check above 
# The sets no_metadata, not_GTEX, check_hypothesis are disjoint
print(set(df_kmers_no_metadata['kmer']).intersection( set(df_kmers_not_in_gtex['kmer'])))
print(set(df_kmers_no_metadata['kmer']).intersection( set(df_check_hypothesis['kmer'])))
print(set(df_kmers_not_in_gtex['kmer']).intersection( set(df_check_hypothesis['kmer'])))

# Drop duplicates
df_kmers_no_metadata = df_kmers_no_metadata.drop_duplicates() 
df_kmers_not_in_gtex = df_kmers_not_in_gtex.drop_duplicates()
df_check_hypothesis = df_check_hypothesis.drop_duplicates()

df_no_info = df_kmers_no_metadata.copy()
df_no_info['info_not_available'] = True

# Augment df with information about GTEX junction presence
df_kmers_not_in_gtex['coordinate_in_GTEX'] = False
df_kmers_no_metadata['coordinate_in_GTEX'] = False # No info
df_coord_in_gtex = pd.concat([df_kmers_not_in_gtex, df_kmers_no_metadata], axis = 0)

# Augment df with information about Novel exon hypothesis
df_kmers_no_metadata.drop('coordinate_in_GTEX', inplace=True, axis=1)
df_kmers_no_metadata['gtexExon2<cancExon2'] = False
df_check_hypothesis['gtexExon2<cancExon2'] = True
df_exon_length = pd.concat([df_check_hypothesis, df_kmers_no_metadata], axis = 0)

filt_meta = filt_meta.merge(df_coord_in_gtex, on = ['kmer', 'junction_coordinate'], how = 'left')
filt_meta['coordinate_in_GTEX'] = filt_meta['coordinate_in_GTEX'].fillna(True)

filt_meta = filt_meta.merge(df_exon_length, on = ['kmer', 'junction_coordinate'], how = 'left')
filt_meta['gtexExon2<cancExon2'] = filt_meta['gtexExon2<cancExon2'].fillna(False)

filt_meta = filt_meta.merge(df_no_info, on = ['kmer', 'junction_coordinate'], how = 'left')
filt_meta['info_not_available'] = filt_meta['info_not_available'].fillna(False)

display(filt_meta.groupby('gtexExon2<cancExon2').count())
display(filt_meta.groupby('coordinate_in_GTEX').count())

set()
set()
{'GKHEERKYT', 'SQQAASKWT', 'SQQAASKAV'}


,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,TCGA25131901A01R156513all,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated,strand,junction_coordinate,coordinate_in_GTEX,info_not_available
gtexExon2<cancExon2,,,,,,,,,,,,,,,,,,,,,
False,131835,676,676,676,676,676,676,131835,131835,131835,...,131835,131835,131835,131835,131835,21,131835,131835,131835,131835
True,127,7,7,7,7,7,7,127,127,127,...,127,127,127,127,127,1,127,127,127,127


,kmer,gtexCohortfilter >0.0,gtexCohortfilter >=1.0,gtexCohortfilter >=2.0,gtexCohortfilter >=3.0,gtexCohortfilter >=5.0,gtexCohortfilter >=10.0,coord,junctionAnnotated,readFrameAnnotated,...,TCGA25131901A01R156513all,TCGA25131301A01R156513all,TCGA61200801A02R156813all,TCGA24143101A01R156613all,TCGA24229801A01R156913all,isAnnotated,strand,junction_coordinate,gtexExon2<cancExon2,info_not_available
coordinate_in_GTEX,,,,,,,,,,,,,,,,,,,,,
False,17230,14,14,14,14,14,14,17230,17230,17230,...,17230,17230,17230,17230,17230,0,17230,17230,17230,17230
True,114732,669,669,669,669,669,669,114732,114732,114732,...,114732,114732,114732,114732,114732,22,114732,114732,114732,114732


In [23]:
path_interest = f'G_{sample_target}_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1.tsv.gz'
path_interest_with_advanced_meta = os.path.join(output_dir, path_interest.replace('.tsv.gz', 'metadata2.tsv.gz'))
print(f'Saving kmers with advanced metadata to {path_interest_with_advanced_meta}')
filt_meta.to_csv(path_interest_with_advanced_meta, sep = '\t', compression = 'gzip', index = None)

Saving kmers with advanced metadata to /cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374/filtering_samples/filters_22March_order_wany_wAnnot/G_TCGA-25-1319-01A-01R-1565-13_SampleLim0.0CohortLimNoneAcrossNone_FiltNormalsGtexCohortCohortlim0.0Across1metadata2.tsv.gz


## Exploratory

In [ ]:
pd.read

### Remove GTEX annotation 

In [19]:
# path_annot = glob.glob('/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/GTEX2019_eth/GTEX2019_6920432_ANNOT_conf2_RFall_ref/cohort_mutNone/*/ref_annot_kmer.gz')

# print(len(path_annot))

In [20]:
# for idx, annot in enumerate(path_annot):
#     kmers_filter_pipeline = len(kmer_post_filter)
#     annot = pd.read_csv(annot, sep = '\t')
#     annot = set(annot['kmer'])
#     kmer_post_filter = kmer_post_filter.difference(annot)
#     if kmers_filter_pipeline != len(kmer_post_filter):
#         print(path_annot[idx], len(kmer_post_filter))